<a href="https://colab.research.google.com/github/sergeiissaev/Recursion-Cellular-Image-Classification/blob/master/Recursion_Cellular_Image_Classification_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Remember to put runtime to TPU!

In [0]:
#imports

import json
import os
import sys
import tensorflow as tf






In [0]:
#import rxrx1-utils
if 'google.colab' in sys.modules:
    !git clone https://github.com/recursionpharma/rxrx1-utils
    sys.path.append('/content/rxrx1-utils')

    from google.colab import auth
    auth.authenticate_user()
    
from rxrx.main import main

fatal: destination path 'rxrx1-utils' already exists and is not an empty directory.
